# Objetivo

Avaliar se há clusters no portfólio 2. Em um segundo momento, a partir de uma clusterização global (na base toda), buscar novos clientes na base completa.

# Método

A partir do conjunto reduzido de features (47), aplicar um processo de clusterização (AgglomerativeClustering).

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import feather

## Leitura da lista de features a excluir

In [2]:
# obtida em EDA Pandas_profiling_sample_10k_market.ipynb
exclude_df = pd.read_csv('../exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)

143

## Carregamento Portfólio 1

In [3]:
p2 = pd.read_csv('../estaticos_market.csv', nrows=1500)
p2.info()

FileNotFoundError: [Errno 2] File b'../estaticos_market.csv' does not exist: b'../estaticos_market.csv'

## Redução no número de features

In [25]:
p2.drop(columns=exclude_variables, inplace= True)

In [26]:
p2.shape

(1500, 49)

# Clusterização

## Dados do portfólio

Para usar nos dados do portfólio, temos que passar as strings para valor numérico.

In [51]:
# portfolio 2, reduzido (micro): p1micro
p2micro = p2.drop(columns=['Unnamed: 0', 'id'])

In [52]:
p2micro.dtypes == object

fl_matriz                                False
de_natureza_juridica                      True
sg_uf                                     True
natureza_juridica_macro                   True
de_ramo                                   True
setor                                     True
idade_empresa_anos                       False
idade_emp_cat                             True
fl_me                                    False
fl_sa                                    False
fl_epp                                   False
fl_mei                                   False
fl_ltda                                  False
dt_situacao                               True
fl_st_especial                           False
fl_email                                 False
fl_telefone                              False
fl_rm                                     True
nm_divisao                                True
nm_segmento                               True
fl_spa                                    True
fl_antt      

In [53]:
p2micro.head(3)

fl_matriz           de_natureza_juridica sg_uf natureza_juridica_macro  \
0       True  SOCIEDADE EMPRESARIA LIMITADA    RN  ENTIDADES EMPRESARIAIS   
1       True          EMPRESARIO INDIVIDUAL    PI                  OUTROS   
2       True          EMPRESARIO INDIVIDUAL    AM                  OUTROS   

                              de_ramo             setor  idade_empresa_anos  \
0             INDUSTRIA DA CONSTRUCAO  CONSTRUÇÃO CIVIL           14.457534   
1  SERVICOS DE ALOJAMENTO/ALIMENTACAO           SERVIÇO            1.463014   
2   TRANSPORTE, ARMAZENAGEM E CORREIO           SERVIÇO            7.093151   

  idade_emp_cat  fl_me  fl_sa  ...  qt_socios  qt_socios_pj  \
0       10 a 15  False  False  ...        2.0           0.0   
1         1 a 5  False  False  ...        1.0           0.0   
2        5 a 10  False  False  ...        1.0           0.0   

   idade_media_socios qt_socios_masculino  qt_socios_feminino  \
0                44.0                 2.0                 NaN   
1                27.0                 1.0                 NaN   
2                32.0                 1.0                 NaN   

          de_faixa_faturamento_estimado   de_faixa_faturamento_estimado_grupo  \
0  DE R$ 1.500.000,01 A R$ 4.800.000,00  DE R$ 1.500.000,01 A R$ 4.800.000,00   
1       DE R$ 81.000,01 A R$ 360.000,00       DE R$ 81.000,01 A R$ 360.000,00   
2                      ATE R$ 81.000,00                      ATE R$ 81.000,00   

  vl_faturamento_estimado_aux vl_faturamento_estimado_grupo_aux qt_filiais  
0                   3132172.8                         3132172.8          0  
1                    210000.0                          210000.0          0  
2                     50000.0                           50000.0          0  

[3 rows x 47 columns]

In [54]:
# Vou começar com 4 features: de_natureza_juridica, sg_uf, de_ramo, de_nivel_atividade
#categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo', 'de_nivel_atividade']
categorical_cols = ['fl_passivel_iss','de_faixa_faturamento_estimado','de_saude_rescencia']
for feat in categorical_cols:
    print(p2micro[feat].value_counts())
    print('\n')

True     849
False    641
Name: fl_passivel_iss, dtype: int64


DE R$ 81.000,01 A R$ 360.000,00            877
ATE R$ 81.000,00                           327
DE R$ 360.000,01 A R$ 1.500.000,00         141
DE R$ 1.500.000,01 A R$ 4.800.000,00        42
DE R$ 4.800.000,01 A R$ 10.000.000,00        6
SEM INFORMACAO                               6
DE R$ 10.000.000,01 A R$ 30.000.000,00       6
DE R$ 30.000.000,01 A R$ 100.000.000,00      2
Name: de_faixa_faturamento_estimado, dtype: int64


ACIMA DE 1 ANO    1231
SEM INFORMACAO     111
ATE 1 ANO          109
Name: de_saude_rescencia, dtype: int64




In [55]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [64]:
# apply le on categorical feature columns
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

#portfolio 1 com labelencoder: p1enc
p2enc=pd.DataFrame()

#categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo']
categorical_cols = ['fl_passivel_iss','de_faixa_faturamento_estimado','de_saude_rescencia']
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
# IMPORTANTE aqui, só três features, para poder visualizar em um gráfico 3D.
# Também porque de_nivel_atividade tem alguns NaN, que tem que ser resolvidos depois
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
p2enc[categorical_cols] = p2micro[categorical_cols].apply(lambda col: le.fit_transform(col))

p2enc[categorical_cols].head(5)

TypeError: ('argument must be a string or number', 'occurred at index de_faixa_faturamento_estimado')

In [57]:
p2enc.shape

(0, 0)

In [48]:
from sklearn.cluster import KMeans

clustering = KMeans(n_clusters=5).fit(p2enc)
labels = clustering.labels_
p2enc['cluster'] = labels

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)


print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)



Estimated number of clusters: 5
Estimated number of noise points: 0


In [50]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
#p2enc['de_natureza_juridica']=p2enc['de_natureza_juridica'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
#p2enc['de_ramo']=p2enc['de_ramo'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
#p2enc['sg_uf']=p2enc['sg_uf'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
#p2enc.head(5)
#['fl_passivel_iss','de_faixa_faturamento_estimado','de_saude_rescencia']
import plotly.express as px
fig = px.scatter_3d(p2enc, x=categorical_cols[1], y=categorical_cols[2], z=categorical_cols[3], color = 'cluster')
fig.show()

In [39]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
p2enc['de_natureza_juridica']=p2enc['de_natureza_juridica'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['de_ramo']=p2enc['de_ramo'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['sg_uf']=p2enc['sg_uf'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc.head(5)

de_natureza_juridica     sg_uf    de_ramo  cluster
0              0.852603 -0.143055  -0.244926        0
1             12.147162  4.096763   0.111031        0
2             24.794295  2.947905  19.031795        3
3             11.847822  0.988321   0.215425        0
4             24.055814  4.230985  14.123266        3

In [40]:
import plotly.express as px
fig = px.scatter_3d(p2enc, x='de_natureza_juridica', y='sg_uf', z='de_ramo', color = 'cluster')
fig.show()

## Conclusão até o momento

Clusterização parece interessante. Observar que, das três categorias avaliadas, o `de_ramo` não teve muita influência, porque está bem polarizado em uma das classes. Próximos passos: incluir mais features (sabendo que vai impedir a visualização em mais de 3 dimensões) e aplicar redução de dimensionalidade (PCA? antes ou depois da clusterização?).